In [1]:
import sys
import os
import numpy as np
import pandas as pd

import h5py

import matplotlib.pyplot as plt

from scipy.stats import spearmanr, pearsonr

import seaborn as sns

from scipy.stats import norm

from basenji.gene import gtf_kv
from basenji import gene as bgene

import pybedtools

from sklearn.metrics import average_precision_score, roc_auc_score

#Helper function to make a bedtool object from the crispr dataframe
def _make_crispr_bedt(crispr_df, allow_other_exons_to_overlap=False, window_size=2000, extra_padding=64):
  
    bed_lines = []
    
    #Loop over rows in crispr dataframe
    for _, row in crispr_df.iterrows() :
        
        chrom = row['chr']
        midpoint = (row['start'] + row['end']) // 2
        start = midpoint - window_size // 2 - extra_padding
        end = midpoint + window_size // 2 + extra_padding
        
        #Construct unique enhancer id
        enh_id = row['chr'] + '_' + str(row['start']) + '_' + str(row['end'])
        if allow_other_exons_to_overlap :
            enh_id += '_' + row['gene']
        
        gene = row['gene']
        
        #Write to bed file
        bed_lines.append('%s %d %d %s %s' % (chrom, start, end, enh_id, gene))
    
    #Create bedtool object
    bedt = pybedtools.BedTool('\n'.join(bed_lines), from_string=True)
    
    return bedt

#Helper function to compute gradient/ISM scores for each enhancer-gene pair in the crispr dataframe
def _score_sites_borzoi(crispr_df, gene_i, grads_ref, seq_starts, seq_ends, elen_ext=3, extra_padding=1064, normalize=True, use_abs=True, use_gaussian=False, sigma=300) :
    
    #Get sequence length (of attributions)
    seq_len = grads_ref.shape[1]
    
    #Initialize scores and metadata
    enh_ref = np.zeros(crispr_df.shape[0])
    enh_seq_starts = np.zeros(crispr_df.shape[0], dtype='int32')
    enh_seq_ends = np.zeros(crispr_df.shape[0], dtype='int32')
    enh_valids = np.zeros(crispr_df.shape[0], dtype='int32')
    
    #Define smoothing helpers
    if use_gaussian :
        window = np.arange(-4*sigma, 4*sigma+1)
        weights = norm.pdf(window, scale=sigma)

    #Loop over enhancer rows in crispr dataframe
    for ei, enh_row in crispr_df.iterrows() :
        
        #Get gene row in attribution score array
        gi = gene_i[enh_row['gene']]
        
        #Get sequence start/end coordinates for attribution scores
        enh_seq_starts[ei] = seq_starts[gi]
        enh_seq_ends[ei] = seq_ends[gi]

        #Extension boundaries
        emid = (enh_row['start'] + enh_row['end']) // 2
        enh_start = emid - elen_ext//2
        enh_end = enh_start + elen_ext

        #Map enhancer coordinates to gene sequence
        eg_start = enh_start - enh_seq_starts[ei]
        eg_end = enh_end - enh_seq_starts[ei]
        eg_mid = emid - enh_seq_starts[ei]

        #Check if enhancer is within sequence bounds
        if eg_start < extra_padding :
            pass #Enhancer outside of sequence
        elif eg_end > seq_len - extra_padding :
            pass #Enhancer outside of sequence
        else :
            
            #Determine smoothing window (if gaussian smoothing is on)
            if use_gaussian :
                
                eg_window = window + eg_mid
                wstart, wend = 0, seq_len
                
                if eg_window[0] < 0:
                    wstart = -eg_window[0]
                if eg_window[-1] > seq_len:
                    wend = seq_len - eg_window[-1] - 1
                eg_window = eg_window[wstart:wend]
          
            #Mark enhancer as valid
            enh_valids[ei] = 1

            #Compute score (gaussian)
            if use_gaussian :
                enh_ref[ei] = np.sum(weights[wstart:wend] * np.abs(grads_ref[gi].astype('float32'))[eg_window]) / np.sum(weights[wstart:wend])
            else : #Otherwise compute sum score
                #Optionally use absolute value of attribution scores
                if use_abs :
                    enh_ref[ei] = np.abs(grads_ref[gi,eg_start:eg_end]).sum(dtype='float32')
                else :
                    enh_ref[ei] = grads_ref[gi,eg_start:eg_end].sum(dtype='float32')

            #Optionally normalize scores
            if normalize:
                enh_ref[ei] /= np.abs(grads_ref[gi]).mean(dtype='float32')

    return enh_ref, enh_seq_starts, enh_seq_ends, enh_valids

#Helper function to remove gene id suffix
def _trim_dot(gene_id):
  
    #Find suffix start
    dot_i = gene_id.rfind('.')
    
    #Attempt to trim gene id
    if dot_i == -1:
        return gene_id
    
    return gene_id[:dot_i]

#Function to compute attribution scores for crispr dataset
def compute_scores_borzoi(satg_h5f, crispr_gtf_file, crispr_table_tsv, score_col='grads', normalize=True, use_abs=True, use_gaussian=False, sigma=300, tpm_file=None, min_tpm_rank=0., exons_gtf=None, allow_other_exons_to_overlap=True) :
    
    #Read sequences and reference attribution scores
    with h5py.File(satg_h5f, 'r') as satg_h5 :
        
        #Get sequence start/.end coordinates
        seq_starts = satg_h5['start'][:]
        seq_ends = satg_h5['end'][:]

        #Load gene ids and map gene id to row index
        gene_ids = [_trim_dot(gene_id.decode('UTF-8')) for gene_id in satg_h5['gene']]
        geneid_i = dict(zip(gene_ids, np.arange(len(gene_ids))))

        #Initialize attribution score structure
        num_seqs, seq_len, _, num_targets = satg_h5[score_col].shape
        grads_ref = np.zeros((num_seqs,seq_len), dtype='float16')
        
        #Loop over sequences
        for si in range(num_seqs) :

            #Get scores
            grads_si = satg_h5[score_col][si].sum(axis=-1)

            #Get sequences and fill masked sequence positions
            seq_1hot = satg_h5['seqs'][si]
            n_mask = (seq_1hot.sum(axis=-1) == 0)
            seq_1hot[n_mask] = np.array([1,0,0,0])
            
            #Slice scores
            grads_ref[si] = grads_si[seq_1hot]

    #Hash gene symbols to indexes
    gene_i = {}
    
    #Loop over lines in crispr gtf file
    for line in open(crispr_gtf_file) :
        kv = gtf_kv(line.split('\t')[8])
        
        #Get gene id base and symbol
        gene_id = _trim_dot(kv['gene_id'])
        gene_symbol = kv['gene_name']
        
        #Fill dictionaries
        if gene_id in geneid_i:
            gene_i[gene_symbol] = geneid_i[gene_id]

    #Score crispr sites
    crispr_df = pd.read_csv(crispr_table_tsv, sep='\t')
    crispr_df['score'], crispr_df['seq_start'], crispr_df['seq_end'], crispr_df['is_valid'] = _score_sites_borzoi(
        crispr_df,
        gene_i,
        grads_ref,
        seq_starts,
        seq_ends,
        normalize=normalize,
        use_abs=use_abs,
        use_gaussian=use_gaussian,
        sigma=sigma
    )

    #Only keep sites within sequence window
    crispr_df = crispr_df.loc[crispr_df['is_valid'] == 1].copy()

    #Optionally load tpm rank dataframe for matched cell type
    if tpm_file is not None :
        tpm_df = pd.read_csv(tpm_file, sep='\t')
        
        #Filter on minimum median TPM rank
        crispr_df = crispr_df.join(tpm_df[['gene_symbol', 'rank_k562_median']].set_index('gene_symbol'), on='gene', how='inner')
        crispr_df = crispr_df.loc[crispr_df['rank_k562_median'] >= min_tpm_rank].copy()
  
    #Optionally load exon annotation and remove crispr loci which overlap exons
    if exons_gtf is not None :

        #Read exons
        transcriptome = bgene.Transcriptome(exons_gtf)

        #Create bedtool object with exons
        exons_bedt = transcriptome.bedtool_exon()

        #Create bedtool object with crispr loci
        crispr_bedt = _make_crispr_bedt(crispr_df, allow_other_exons_to_overlap)

        #Optionally map gene id base to symbol
        gene_symb = None
        if allow_other_exons_to_overlap and tpm_df is not None :
            gene_symb = {}
            for _, row in tpm_df.iterrows() :
                gene_symb[row['gene_base']] = row['gene_symbol']

        #Intersect crispr loci with exon bed
        overlapping_ids = []

        #Loop over overlapping site-exon pairs
        for overlap in exons_bedt.intersect(crispr_bedt, wo=True) :
            exon_gene_id = _trim_dot(overlap[3])
            enh_id = overlap[9]
            enh_gene_symbol = overlap[10]

            #Check if site is overlapping measured gene in crispr dataset
            if gene_symb is not None and exon_gene_id in gene_symb and gene_symb[exon_gene_id] == enh_gene_symbol :
                overlapping_ids.append(enh_id)
            else :
                overlapping_ids.append(enh_id)

        #Get set of overlapping enhancer ids
        overlapping_ids = list(set(overlapping_ids))

        #Get enhancer id column
        crispr_df['enh_id'] = crispr_df['chr'] + '_' + crispr_df['start'].astype(str) + '_' + crispr_df['end'].astype(str)
        if allow_other_exons_to_overlap :
            crispr_df['enh_id'] += '_' + crispr_df['gene'].astype(str)

        #Keep non-overlapping crispr rows only
        crispr_df = crispr_df.loc[~crispr_df['enh_id'].isin(overlapping_ids)].copy()
    
    return crispr_df


In [2]:
#Compute borzoi scores (ism_shuffle; ensemble)

ablation_strs = [
    'baseline',
    'human_all',
    'human_dnase_atac_rna',
    'k562_all',
    'k562_dnase_atac_rna',
    'k562_rna',
    'multisp_dnase_atac_rna',
    'multisp_rna',
    'multisp_no_unet',
]

for ablation_str in ablation_strs :

    #Get crispr dataframe with scores
    crispr_df = compute_scores_borzoi(
        'flowfish_miborzoi_' + ablation_str + '_undo_ism_shuffle_clip/ism_mean.h5',
        'crispr_genes.gtf',
        'crispr_table.tsv',
        score_col='isms',
        normalize=False,
        use_abs=False,
        use_gaussian=False,
        sigma=300,
        tpm_file=None,
        min_tpm_rank=0.,
        exons_gtf=None,
        allow_other_exons_to_overlap=True,
    )

    #Compute percent changes from log fold changes
    crispr_df['pct_change'] = (np.exp(-crispr_df['score']) - 1) * 100
    crispr_df['score'] = np.abs(crispr_df['score'])

    #Store stats
    crispr_df.to_csv("site_scores_flowfish_k562_ism_miborzoi_" + ablation_str + "_ensemble.csv", sep='\t', index=False)


In [3]:
#Compute borzoi scores (ism_shuffle; cross-fold)

ablation_strs = [
    'baseline',
    'human_all',
    'human_dnase_atac_rna',
    'k562_all',
    'k562_dnase_atac_rna',
    'k562_rna',
    'multisp_dnase_atac_rna',
    'multisp_rna',
    'multisp_no_unet',
]

fold_index = [0, 1]

for ablation_str in ablation_strs :

    for fold_ix in fold_index :
    
        #Get crispr dataframe with scores
        crispr_df = compute_scores_borzoi(
            'flowfish_miborzoi_' + ablation_str + '_undo_ism_shuffle_clip/ism_f' + str(fold_ix) + 'c0.h5',
            'crispr_genes.gtf',
            'crispr_table.tsv',
            score_col='isms',
            normalize=False,
            use_abs=False,
            use_gaussian=False,
            sigma=300,
            tpm_file=None,
            min_tpm_rank=0.,
            exons_gtf=None,
            allow_other_exons_to_overlap=True,
        )

        #Compute percent changes from log fold changes
        crispr_df['pct_change'] = (np.exp(-crispr_df['score']) - 1) * 100
        crispr_df['score'] = np.abs(crispr_df['score'])

        #Store stats
        crispr_df.to_csv("site_scores_flowfish_k562_ism_miborzoi_" + ablation_str + "_f" + str(fold_ix) + ".csv", sep='\t', index=False)
